### <span style="color:#800000">NER using spaCy</span>

### <span style="color:#FF00FF">Import libraries</span>

In [1]:
import pandas as pd
import ast
import pickle

import spacy
from spacy import displacy
from spacy.training import Example
from spacy.util import minibatch, compounding

from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

### <span style="color:#FF00FF">Load the dataset</span>

In [2]:
# Reading data and creating the dataframe 
df = pd.read_csv("../data/jobdescriptions_train.csv")

# Checking the first 5 rows of the data
df.head()

,domain,link,title,company,experience,salary,location,description,role,industry_type,qualification,functional_area,employment_type,role_category,skills,clean_description,tags
0,information-technology-jobs,https://www.naukri.com/job-listings-head-infor...,Head - Information Technology,Bright Consultant,15 - 25 years,Not Disclosed,Delhi / NCR,: - manage all it verticals of the company li...,"Head/VP/GM-Technology(IT)/CTO,","Industrial Products, Heavy Machinery,","['UG :Any Graduate in Any Specialization', 'PG...","IT Software - Network Administration, Security,","Full Time, Permanent",Senior Management,"['Network Security', 'Information Security', '...",manage verticals company like applications inf...,"{'entities': [(323, 331, 'DEPT'), (30, 42, 'SK..."
1,information-technology-jobs,https://www.naukri.com/job-listings-urgent-ope...,Urgent Opening - Sr. Executive- IT (informatio...,Ecom Express Private Limited,4 - 9 years,"₹ 2,50,000 - 4,50,000 P.A.",Ahmedabad,"hi, greetings from ecom express pvt ltd!!! we ...","IT/Networking-Manager,","Courier, Transportation, Freight , Warehousing,",['UG :Any Graduate in Any Specialization'],"IT Software - Application Programming, Mainten...","Full Time, Permanent",Admin/Maintenance/Security/Datawarehousing,"['LAN Troubleshooting', 'Hardware Networking',...",hi greetings ecom express pvt ltd urgent openi...,"{'entities': [(394, 411, 'ROLE'), (602, 610, '..."
2,information-technology-jobs,https://www.naukri.com/job-listings-informatio...,Information Technology (IT) & Business Solutio...,Procter & Gamble,8 - 10 years,Not Disclosed,Mumbai,do you want to join our team of women and m...,"Project Manager-IT/Software,","FMCG, Foods, Beverage,","['UG :BCA in Computers, B.Tech/B.E. in Compute...","IT Software - Application Programming, Mainten...","Full Time, Permanent",Project Management,"['Application Development', 'Agile Project Man...",want join team women men professionals apply s...,"{'entities': [(2005, 2021, 'DEPT'), (1954, 197..."
3,information-technology-jobs,https://www.naukri.com/job-listings-senior-sol...,Senior Solution Manager - Information Technology,Procter & Gamble,8 - 12 years,Not Disclosed,Mumbai,"solutions manager leads product roadmap, ...","IT/Networking-Manager,","FMCG, Foods, Beverage,","['UG :Any Graduate in Any Specialization', 'PG...","IT Software - Application Programming, Mainten...","Full Time, Permanent",Admin/Maintenance/Security/Datawarehousing,"['Agile Methodology', 'Fund Management', 'Prod...",solutions manager leads product roadmap create...,"{'entities': [(893, 913, 'SKILL'), (1061, 1075..."
4,information-technology-jobs,https://www.naukri.com/job-listings-informatio...,Information Technology,Stefanini,12 - 22 years,Not Disclosed,Noida,we are looking for someone who is passionat...,"System Administrator,","Recruitment, Staffing,","['UG :Any Graduate in Any Specialization', 'PG...","IT Software - Application Programming, Mainten...","Full Time, Permanent",Admin/Maintenance/Security/Datawarehousing,"['SAN', 'Automation', 'Debugging', 'NetBackup'...",looking someone passionate emerging technologi...,"{'entities': [(3277, 3297, 'SKILL'), (2428, 24..."


### <span style="color:#FF00FF">Training spaCy NER model with Custom Entities</span>

In [3]:
TRAIN_DATA = df[['clean_description','tags']].to_numpy()

In [4]:
# Import and load the spacy model
nlp=spacy.load("en_core_web_sm") 

# Getting the ner component
ner=nlp.get_pipe('ner')

# Adding labels to the `ner`
ner.add_label('SKILL')
ner.add_label('ROLE')
ner.add_label('DEPT')

# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [5]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        nlp.add_pipe('ner', last=True)
       

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            #random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                examples = []
                for text, annots in batch:
                    annots = ast.literal_eval(annots)
                    examples.append(Example.from_dict(nlp.make_doc(text), annots))
                    nlp.update(examples,losses=losses)
                print("Losses", losses) 
    return nlp

In [6]:
jdnlp = train_spacy(TRAIN_DATA, iterations = 20)

Statring iteration 0
Losses {'ner': 1342.587233364582}
Losses {'ner': 2743.3319696187973}
Losses {'ner': 3096.3558551976603}
Losses {'ner': 3248.51164596506}
Losses {'ner': 3485.070929487367}
Losses {'ner': 3577.0816759230947}
Losses {'ner': 3672.19098782129}
Losses {'ner': 3848.342194174955}
Losses {'ner': 4037.342335229405}
Losses {'ner': 4217.378909881765}
Losses {'ner': 4223.253225199043}
Losses {'ner': 4586.481605743429}
Losses {'ner': 4806.066182743514}
Losses {'ner': 4999.260286738764}
Losses {'ner': 5206.701077337718}
Losses {'ner': 5378.515542241307}
Losses {'ner': 5519.624816685284}
Losses {'ner': 5678.549432656981}
Losses {'ner': 5964.945157788162}
Losses {'ner': 6156.00424803289}
Losses {'ner': 6259.221911209092}
Losses {'ner': 6345.93847689692}
Losses {'ner': 6425.6806435405815}
Losses {'ner': 6530.002227845231}
Losses {'ner': 6581.830454928604}
Losses {'ner': 6651.495613972318}
Losses {'ner': 6699.395460488064}
Losses {'ner': 6733.735351793318}
Losses {'ner': 7042.6800613

Losses {'ner': 22206.325484215537}
Losses {'ner': 22206.38918000078}
Losses {'ner': 22243.784997221555}
Losses {'ner': 22371.97039530121}
Losses {'ner': 22723.369006428547}
Losses {'ner': 22845.282118808358}
Losses {'ner': 22908.702955355882}
Losses {'ner': 22983.025613905676}
Losses {'ner': 23037.91380912386}
Losses {'ner': 23327.31329493604}
Losses {'ner': 23427.833751988717}
Losses {'ner': 23509.217903332705}
Losses {'ner': 23741.39615825842}
Losses {'ner': 23804.741285641558}
Losses {'ner': 23872.600346320407}
Losses {'ner': 23979.758151142294}
Losses {'ner': 24062.726005040236}
Losses {'ner': 24129.631004898525}
Losses {'ner': 24240.236282358655}
Losses {'ner': 24263.37152079195}
Losses {'ner': 24330.549556522114}
Losses {'ner': 24621.31388380903}
Losses {'ner': 24644.097635435373}
Losses {'ner': 24778.3135109937}
Losses {'ner': 24799.853065368374}
Losses {'ner': 24984.85482697691}
Losses {'ner': 25070.749004105925}
Losses {'ner': 25279.459913534964}
Losses {'ner': 25407.846304736

Losses {'ner': 12175.294177621501}
Losses {'ner': 12183.642427898209}
Losses {'ner': 12194.38608746287}
Losses {'ner': 12261.455792745626}
Losses {'ner': 12289.762946933957}
Losses {'ner': 12317.181416179084}
Losses {'ner': 12337.111749583899}
Losses {'ner': 12360.880948768283}
Losses {'ner': 12406.524340296224}
Losses {'ner': 12480.129149877695}
Losses {'ner': 12528.046310062342}
Losses {'ner': 12603.010383316798}
Losses {'ner': 12645.457340349443}
Losses {'ner': 12718.202847287072}
Losses {'ner': 12745.728090210223}
Losses {'ner': 12823.92232329899}
Losses {'ner': 12877.24862485752}
Losses {'ner': 12927.994478640776}
Losses {'ner': 12963.38930690668}
Losses {'ner': 12982.365455294186}
Losses {'ner': 13015.88488397827}
Losses {'ner': 13099.555506476714}
Losses {'ner': 13121.161000962382}
Losses {'ner': 13176.373758159563}
Losses {'ner': 13205.84397271147}
Losses {'ner': 13263.36367336471}
Losses {'ner': 13292.102254936835}
Losses {'ner': 13314.800516248531}
Losses {'ner': 13407.105159

Losses {'ner': 8743.80310321634}
Losses {'ner': 8753.147487495547}
Losses {'ner': 8765.792435110863}
Losses {'ner': 8773.700318381443}
Losses {'ner': 8801.490113932487}
Losses {'ner': 8802.56667469233}
Losses {'ner': 8840.921455961397}
Losses {'ner': 8841.980709713012}
Losses {'ner': 8867.978470466756}
Losses {'ner': 8919.996658311611}
Losses {'ner': 8959.095049989015}
Losses {'ner': 8994.890498955205}
Losses {'ner': 8998.801388518916}
Losses {'ner': 9003.416356025653}
Losses {'ner': 9033.19398326519}
Losses {'ner': 9039.885609884068}
Losses {'ner': 9066.899025422143}
Losses {'ner': 9076.461709817879}
Losses {'ner': 9111.001058995244}
Losses {'ner': 9138.943796857626}
Losses {'ner': 9141.609777315825}
Losses {'ner': 9150.019551588799}
Losses {'ner': 9185.070625400542}
Losses {'ner': 9204.737722647007}
Losses {'ner': 9223.486365634944}
Losses {'ner': 9233.802143576359}
Losses {'ner': 9239.39855253283}
Losses {'ner': 9267.268247232998}
Losses {'ner': 9292.066938390391}
Losses {'ner': 929

Losses {'ner': 5111.669490962319}
Losses {'ner': 5118.702278620499}
Losses {'ner': 5146.126444557542}
Losses {'ner': 5183.3529844936465}
Losses {'ner': 5203.384178665903}
Losses {'ner': 5223.596673980292}
Losses {'ner': 5231.606337122792}
Losses {'ner': 5242.041051285384}
Losses {'ner': 5272.023289344936}
Losses {'ner': 5338.860314912484}
Losses {'ner': 5392.99624638411}
Losses {'ner': 5412.297228194542}
Losses {'ner': 5432.147513685288}
Losses {'ner': 5452.1277284193775}
Losses {'ner': 5455.519896026227}
Losses {'ner': 5463.314610181439}
Losses {'ner': 5480.5494453399915}
Losses {'ner': 5521.3106069881105}
Losses {'ner': 5546.781824975957}
Losses {'ner': 5588.81141646703}
Losses {'ner': 5630.414348741553}
Losses {'ner': 5676.899202238712}
Losses {'ner': 5723.743781557922}
Losses {'ner': 5834.793567405397}
Losses {'ner': 5876.818308591127}
Losses {'ner': 6005.03555111987}
Losses {'ner': 6137.588147491262}
Losses {'ner': 6171.0473223511935}
Losses {'ner': 6266.360767801994}
Losses {'ner

...

Losses {'ner': 730.8278757960256}
Losses {'ner': 770.0820568547192}
Losses {'ner': 782.0060428161266}
Losses {'ner': 816.8040745611466}
Losses {'ner': 840.3502483987386}
Losses {'ner': 850.1911149619377}
Losses {'ner': 851.3206229246247}
Losses {'ner': 859.4742238916069}
Losses {'ner': 860.2099797306213}
Losses {'ner': 860.8850670303918}
Losses {'ner': 864.5644718944496}
Losses {'ner': 864.7663637862319}
Losses {'ner': 864.7902622958123}
Losses {'ner': 868.4720995248352}
Losses {'ner': 879.8851412213818}
Losses {'ner': 885.2375081786696}
Losses {'ner': 1035.5786325343106}
Losses {'ner': 1117.7574691515188}
Losses {'ner': 1141.4788737282977}
Losses {'ner': 1143.4853148184238}
Losses {'ner': 1154.219336958066}
Losses {'ner': 1185.7174670360596}
Losses {'ner': 1197.0200744901995}
Losses {'ner': 1209.3428668129625}
Losses {'ner': 1215.3103885605253}
Losses {'ner': 1236.3891235111728}
Losses {'ner': 1248.3440333254543}
Losses {'ner': 1253.657257232063}
Losses {'ner': 1255.3381763252332}
Los

Losses {'ner': 222.87232776720165}
Losses {'ner': 224.7407863632905}
Losses {'ner': 236.75013900741143}
Losses {'ner': 258.4246452044416}
Losses {'ner': 260.69885012944}
Losses {'ner': 277.8944266036202}
Losses {'ner': 282.0147343084248}
Losses {'ner': 283.6707063753476}
Losses {'ner': 286.4042039454871}
Losses {'ner': 289.3593776102396}
Losses {'ner': 289.359506935211}
Losses {'ner': 289.359506955467}
Losses {'ner': 289.35950720090227}
Losses {'ner': 296.3192602512522}
Losses {'ner': 296.3192602701674}
Losses {'ner': 304.28476526900755}
Losses {'ner': 312.3282641328212}
Losses {'ner': 335.51409854205406}
Losses {'ner': 335.6353342091093}
Losses {'ner': 335.63533420911625}
Losses {'ner': 338.3736932702859}
Losses {'ner': 338.3736945583802}
Losses {'ner': 338.3736945583813}
Losses {'ner': 338.7459441904103}
Losses {'ner': 346.6682503675022}
Losses {'ner': 352.44104396173276}
Losses {'ner': 354.41310994227075}
Losses {'ner': 363.57674521588024}
Losses {'ner': 372.8377845015011}
Losses {'

Losses {'ner': 2393.9276041712387}
Losses {'ner': 2411.569816356419}
Losses {'ner': 2411.5843739270276}
Losses {'ner': 2416.0858274134116}
Losses {'ner': 2416.0858295732305}
Losses {'ner': 2425.9732739448064}
Losses {'ner': 2427.340267995439}
Losses {'ner': 2444.640386096258}
Losses {'ner': 2451.915814404004}
Losses {'ner': 2453.9313624727283}
Losses {'ner': 2459.519170938931}
Losses {'ner': 2460.011023486438}
Losses {'ner': 2461.1336256719865}
Losses {'ner': 2465.3876685591504}
Losses {'ner': 2478.4112267129153}
Losses {'ner': 2482.434230728316}
Losses {'ner': 2482.4417379116812}
Losses {'ner': 2482.4432302038713}
Losses {'ner': 2485.582874204151}
Losses {'ner': 2505.6241199541355}
Losses {'ner': 2508.933784500595}
Losses {'ner': 2509.1357825725568}
Statring iteration 18
Losses {'ner': 12.845374893588925}
Losses {'ner': 21.124155909500082}
Losses {'ner': 32.25590091826171}
Losses {'ner': 39.084600631910654}
Losses {'ner': 66.34199366599384}
Losses {'ner': 70.29767017216278}
Losses {'n

Losses {'ner': 2080.9082230188433}
Losses {'ner': 2082.7758937390245}
Losses {'ner': 2094.747489923205}
Losses {'ner': 2095.0241132193014}
Losses {'ner': 2110.9352001038806}
Losses {'ner': 2121.063201566356}
Losses {'ner': 2123.242412814212}
Losses {'ner': 2127.4926253805265}
Losses {'ner': 2130.218840193404}
Losses {'ner': 2131.793313171446}
Losses {'ner': 2135.712316455544}
Losses {'ner': 2146.8887558105303}
Losses {'ner': 2147.5868899377883}
Losses {'ner': 2150.8382017543395}
Losses {'ner': 2150.935600835346}
Losses {'ner': 2151.27416858787}
Losses {'ner': 2153.3336115395196}
Losses {'ner': 2156.5221710297324}
Losses {'ner': 2158.6333270732184}
Losses {'ner': 2161.441228430973}
Losses {'ner': 2178.5955822162873}
Losses {'ner': 2178.697403789216}
Losses {'ner': 2182.8677655208053}
Losses {'ner': 2182.8677655464376}
Losses {'ner': 2185.9010330526403}
Losses {'ner': 2186.9944871351713}
Losses {'ner': 2204.7466724443307}
Losses {'ner': 2210.5744107306696}
Losses {'ner': 2212.66221732316

Losses {'ner': 1741.7896025587968}
Losses {'ner': 1741.7969495836912}
Losses {'ner': 1741.8840840523953}
Losses {'ner': 1741.904301255108}
Losses {'ner': 1763.8543262836238}
Losses {'ner': 1772.2635944861026}
Losses {'ner': 1784.1990045214482}
Losses {'ner': 1784.2109672483361}
Losses {'ner': 1784.4665228923345}
Losses {'ner': 1787.070072002353}
Losses {'ner': 1797.7219854281134}
Losses {'ner': 1797.7553292960413}
Losses {'ner': 1797.7716204396374}
Losses {'ner': 1797.7716451671188}
Losses {'ner': 1801.770104419793}
Losses {'ner': 1811.0520650205158}
Losses {'ner': 1811.0631261008718}
Losses {'ner': 1825.542574839994}
Losses {'ner': 1826.0735175350055}
Losses {'ner': 1826.0740119632383}
Losses {'ner': 1826.0784654231986}
Losses {'ner': 1826.0784717164888}
Losses {'ner': 1826.0784834972517}
Losses {'ner': 1826.0784843985443}
Losses {'ner': 1826.0784855275106}
Losses {'ner': 1826.087461349909}
Losses {'ner': 1842.0708160002632}
Losses {'ner': 1848.3532463222593}
Losses {'ner': 1857.44985

### <span style="color:#FF00FF">Model Evaluation</span>

In [7]:
TEST_DATA = df[['clean_description','tags']].sample(frac=1).reset_index(drop=True)

In [8]:
def get_predvalues(pred,text):
    res = "O"
    for i in pred:
        if i[1] == text:
            res = "B-"+i[2]
    return res

test = []
for i, j in TEST_DATA.iterrows():
    annots = ast.literal_eval(j[1])
    text   = j[0]
    pred = []    
    doc = jdnlp(text)
    for ent in doc.ents:
        pred.append((i,ent.text, ent.label_))
        
        
    for k in annots.get('entities'):
        st, ed = k[0],k[1]
        txt = text[st:ed]
        tag = get_predvalues(pred,txt)
        test.append((i,txt,"B-"+k[2],tag))

In [9]:
cols = ["sentence_id","text","actual","pred"]
tdf = pd.DataFrame(test,columns=cols)

y_test = tdf["actual"].tolist()
y_pred = tdf["pred"].tolist()

y_test = [[ind] for ind in y_test]
y_pred = [[ind] for ind in y_pred]

In [10]:
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred) 
recall =  recall_score(y_test,y_pred)
f1score = f1_score(y_test,y_pred)

SPACY_metrics = {
    "F1-score" : f1score,
    "Accuracy" : accuracy,
    "Precision" : precision,
    "Recall" : recall
}
print(SPACY_metrics)

{'F1-score': 0.9809441731303264, 'Accuracy': 0.9626914763650033, 'Precision': 0.9999023914104441, 'Recall': 0.9626914763650033}


### <span style="color:#FF00FF">Classification report</span>

In [11]:
class_report = classification_report(y_test,y_pred, digits=4)
print(class_report)

              precision    recall  f1-score   support

        DEPT     1.0000    0.9374    0.9677      2748
        ROLE     0.9995    0.9781    0.9887      1965
       SKILL     1.0000    0.9693    0.9844      5928

   micro avg     0.9999    0.9627    0.9809     10641
   macro avg     0.9998    0.9616    0.9803     10641
weighted avg     0.9999    0.9627    0.9809     10641



### <span style="color:#FF00FF">Save Model and metrics</span>

In [12]:
SPACY_objects = {
    "spacy_metrics" : SPACY_metrics,
    "spacy_model" : jdnlp
}

pickle_out = open( "../models/SPACY_objects.pkl", "wb" )
pickle.dump(SPACY_objects, pickle_out)

### <span style="color:#FF00FF">Load Model and Test</span>

In [13]:
pickle_in = open("../models/SPACY_objects.pkl", "rb" )
SPACY_obj = pickle.load(pickle_in)

### <span style="color:#FF00FF">Predict on new JD</span>

In [14]:
jdnlp = SPACY_obj.get("spacy_model")

# Testing the model
JD = """description scope role piramal pharma solutions pps chief information officer cio role provide vision leadership developing implementing information technology initiatives align vision piramal pharma solutions pps businesses pps cio charter build competitive edge business proactively building world class high quality innovative technology digital analytics solutions global operations job overview strategy innovation strategic business partner create enhanced digital technology vision enterprise identify opportunities differentiated technology capabilities solutions p roactively recommend solutions business functional leadership team considering business vision industry trends bringing outside perspective p ush bar technology innovation imbibing cutting edge technological innovations global benchmarks blue sky thinking create user friendly technologies offering great experience cts champion change agent accelerating organizational changes required create sustain enterprise technology capabilities cts thought leader emerging digital business models technologies articulating digital future enterprise role internally externally enable business growth enable inorganic business growth merger acquisition leading due diligence driving integration post acquisition lead strategic operational planning implementation achieve business goals fostering innovation prioritizing initiatives coordinating evaluation deployment management current future systems across organization coordinate facilitate consultation relevant business stakeholders define business systems requirements new technology implementations planning execution partner various site ho teams manage project portfolio relate selection acquisition development implementation major information systems defines governance mechanism metrics review progress technology projects business case achievement technology budget company provide upfront estimates costs various heads keep track spends ensure best roi company technology investment maintaining balance frugality financial discipline adequately gearing future growth b uild ecosystem group technology teams partners including startups product vendors develop implement technology solutions b uild future read technology team attracting retaining upskilling industry best talent compliance information security collaborate piramal quality e compliance qec team ensure quality compliance per defined sops guidelines accordance 21 cfr part 11 gamp guidelines collaborate information security team ensure adherence information security guidelines processes skills abilities exceptional inter personal skills enabling engagement levels across leadership skills including ability manage large team understanding strategy business technology application levels environment priorities goals quickly change evolve also skills think strategically including developing information security strategies interpreting handling complex information acting political sensitivity driving engaging positively change sound understanding portfolio program project management track record delivering enabling large scale complex change programs qualifications experience delivering strategy delivery essential across multiple organisations desirable experience working pharma business senior position essential experience procuring managing large complex outcome based contracts interdependencies experience working senior management team develop business focussed strategies effectively support business needs experience technologies sap salesforce bi pharma quality applications preferred experience digital technologies ar vr rpa chatbots ai ml etc preferred experience joint procurement market testing outsourcing well negotiating quality cost effective services experience successfully implementing strategy business planning evidence delivering high quality customer focussed services experience contributing development implementation effective management information systems aid decision making process evidence contribution major transformation building teams time change"""

doc = jdnlp(JD)
print("Entities", set([(ent.text, ent.label_) for ent in doc.ents]))

colors = {"SKILL":"#FFF380","ROLE":"#8EEBEC","DEPT":"#E7A1B0"}
options = {'ents': ['SKILL', 'ROLE','DEPT'], 'colors':colors}
displacy.render(doc,style='ent',jupyter=True, options=options)

Entities {('leadership', 'ROLE'), ('track record', 'SKILL'), ('lead', 'ROLE'), ('compliance', 'ROLE'), ('deployment', 'SKILL'), ('pharma', 'DEPT'), ('application', 'SKILL'), ('information systems', 'SKILL'), ('bi', 'SKILL'), ('information security', 'DEPT'), ('senior management', 'ROLE'), ('support', 'SKILL'), ('sap', 'SKILL'), ('salesforce', 'SKILL'), ('applications', 'SKILL'), ('development', 'SKILL'), ('information technology', 'DEPT'), ('operations', 'DEPT'), ('developing', 'SKILL'), ('testing', 'SKILL'), ('project management', 'ROLE'), ('decision making', 'SKILL'), ('budget', 'SKILL'), ('analytics', 'DEPT')}
